# Business Data Science

Abhilash Vikram Gupta

https://www.kaggle.com/c/mis382n-fall-2019

In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, SelectKBest, chi2
from sklearn.model_selection import train_test_split

from keras import Sequential
from keras.layers import Dense
from keras.callbacks import Callback, EarlyStopping

from IPython.display import display

%matplotlib inline

In [21]:
df = pd.read_csv('data/train_final.csv', index_col='Id')
df_ult = pd.read_csv('data/test_final.csv', index_col='Id')

X = df.drop('Y', axis='columns').values
y = df['Y'].values

n_features = X.shape[1]

X_ult = df_ult.values

weight_1 = len(y[y == 1])/len(y)

sc = StandardScaler()
X_std = sc.fit_transform(X)
X_std

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2)

In [14]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(16, activation='softmax', kernel_initializer='random_normal', input_dim=24))
classifier.add(Dense(8, activation='softmax', kernel_initializer='random_normal', input_dim=16))
classifier.add(Dense(4, activation='softmax', kernel_initializer='random_normal', input_dim=8))
classifier.add(Dense(1, activation='softmax', kernel_initializer='random_normal', input_dim=4))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

classifier.compile(optimizer= 'adam',loss= 'binary_crossentropy', metrics= ['accuracy'])

classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 2         
Total params: 579
Trainable params: 579
Non-trainable params: 0
_________________________________________________________________


In [31]:
# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

# model & train

model = Sequential()

model.add(Dense(16, activation='softmax', kernel_initializer='random_normal', input_dim=24))
model.add(Dense(8, activation='softmax', kernel_initializer='random_normal', input_dim=16))
model.add(Dense(4, activation='softmax', kernel_initializer='random_normal', input_dim=8))
model.add(Dense(2, activation='softmax', kernel_initializer='random_normal', input_dim=4))
model.add(Dense(1, activation='softmax', kernel_initializer='random_normal', input_dim=2))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc_roc])

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

model.fit(X, y, 
          validation_split=0.2, 
          shuffle=True, 
          batch_size=32, 
          epochs=100, 
          callbacks=my_callbacks, 
          class_weight={0: 1 - weight_1, 1: weight_1});

Train on 13106 samples, validate on 3277 samples
Epoch 1/100
13106/13106 [==============================] - 5s 363us/step - loss: 0.4784 - auc_roc: 0.5089 - val_loss: 0.3548 - val_auc_roc: 0.4963
Epoch 2/100
13106/13106 [==============================] - 4s 275us/step - loss: 0.2793 - auc_roc: 0.4998 - val_loss: 0.2164 - val_auc_roc: 0.4969
Epoch 3/100
13106/13106 [==============================] - 3s 225us/step - loss: 0.1767 - auc_roc: 0.4955 - val_loss: 0.1432 - val_auc_roc: 0.4997
Epoch 4/100
13106/13106 [==============================] - 3s 260us/step - loss: 0.1206 - auc_roc: 0.5007 - val_loss: 0.1016 - val_auc_roc: 0.4993
Epoch 5/100
13106/13106 [==============================] - 3s 255us/step - loss: 0.0877 - auc_roc: 0.4990 - val_loss: 0.0764 - val_auc_roc: 0.4996
Epoch 6/100
13106/13106 [==============================] - 4s 320us/step - loss: 0.0672 - auc_roc: 0.4991 - val_loss: 0.0602 - val_auc_roc: 0.4998
Epoch 7/100
13106/13106 [==============================] - 3s 223us/s

13106/13106 [==============================] - 3s 207us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 57/100
13106/13106 [==============================] - 3s 194us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 58/100
13106/13106 [==============================] - 3s 214us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 59/100
13106/13106 [==============================] - 1s 103us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 60/100
13106/13106 [==============================] - 2s 121us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 61/100
13106/13106 [==============================] - 2s 169us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234 - val_auc_roc: 0.5000
Epoch 62/100
13106/13106 [==============================] - 4s 274us/step - loss: 0.0217 - auc_roc: 0.5000 - val_loss: 0.0234

In [29]:
roc_auc_score(y_test, model.predict(sc.transform(X_test)))

0.5